In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('/content/op.csv')
df.columns
df1 = df.iloc[:, 0:21]
df1.dropna(inplace=True)
df1.drop_duplicates(subset="id", keep="first", inplace=True)

<ipython-input-17-1e552ec5ee4b>:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/op.csv')


In [ ]:
df1.shape

(18352, 21)

In [ ]:
audio_features = df1.iloc[:, 0:11]


In [ ]:
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

In [ ]:
scaled_audio_features

array([[-1.63891055,  1.02057652,  0.86898795, ..., -0.30267307,
        -0.92199266, -0.12764474],
       [-1.63875205,  0.32928528, -0.93261248, ..., -0.36045741,
        -0.5083696 , -1.25725872],
       [-1.63859355,  1.13784914,  1.31237248, ..., -0.36043956,
         1.17528837, -1.61916146],
       ...,
       [ 1.77213591, -1.54707667, -0.62392705, ..., -0.36045741,
        -0.4818553 ,  0.46486976],
       [ 1.77229441,  0.73665333, -0.06268081, ..., -0.35191913,
        -0.62768394,  0.37109048],
       [ 1.7724529 ,  0.50210809,  0.97001227, ..., -0.36039747,
        -0.4288267 ,  1.33872207]])

Hierarchical Clustering:

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

# Compute the linkage matrix using Ward's method
Z = linkage(scaled_audio_features, method='ward')


In [ ]:
# Plot a dendrogram to visualize the clustering hierarchy
dendrogram(Z)

DBSCAN Clustering:

In [ ]:
from sklearn.cluster import DBSCAN

# Create a DBSCAN object with the desired parameters
dbscan = DBSCAN(eps=0.5, min_samples=5)

# Fit the model to the data and get the cluster labels
labels = dbscan.fit_predict(scaled_audio_features)

# Add the cluster labels to the original dataframe
df1['cluster'] = labels

# Print the number of songs in each cluster
print(df1['cluster'].value_counts())


-1    18296
 6        9
 2        8
 1        7
 3        7
 0        5
 7        5
 4        5
 5        5
 8        5
Name: cluster, dtype: int64


MeanShift Clustering:

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

# Estimate the bandwidth parameter using the median distance
bandwidth = estimate_bandwidth(scaled_audio_features, quantile=0.2)

# Create a MeanShift object with the estimated bandwidth
meanshift = MeanShift(bandwidth=bandwidth)

# Fit the model to the data and get the cluster labels
labels = meanshift.fit_predict(scaled_audio_features)

# Add the cluster labels to the original dataframe
df1['cluster'] = labels

# Print the number of songs in each cluster
print(df1['cluster'].value_counts())


0    18261
1       64
2       27
Name: cluster, dtype: int64


Spectral Clustering:

In [ ]:
from sklearn.cluster import SpectralClustering

# Create a SpectralClustering object with the desired parameters
spectral = SpectralClustering(n_clusters=10, assign_labels='discretize', random_state=42)

# Fit the model to the data and get the cluster labels
labels = spectral.fit_predict(scaled_audio_features)

# Add the cluster labels to the original dataframe
df1['cluster'] = labels

# Print the number of songs in each cluster
print(df1['cluster'].value_counts())


0    9779
1    6881
9    1662
6       7
8       6
4       5
7       4
5       4
3       2
2       2
Name: cluster, dtype: int64


Evaluating clustering solutions- Sum of the Squared Error:

In [ ]:
model = KMeans(n_clusters=100)
# Fit the model to the data
model.fit(scaled_audio_features)

# Get the cluster labels for each data point
labels = model.labels_

# Add the cluster labels to the original dataframe
df1['cluster'] = labels

# Print the number of songs in each cluster
print(df1['cluster'].value_counts())

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


28    352
75    323
84    321
61    310
26    298
     ... 
71     89
19     85
60     82
63     78
54     71
Name: cluster, Length: 100, dtype: int64


In [ ]:
# Calculate the SSE for the clustering solution
sse = 0
for i in range(100):
    cluster_points = scaled_audio_features[labels == i]
    centroid = model.cluster_centers_[i]
    sse += np.sum((cluster_points - centroid) ** 2)

# Print the SSE
print("SSE for the clustering solution:", sse)


SSE for the clustering solution: 57456.24200521049


In [ ]:
audio_features = df1.iloc[:, 0:11].values
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)
similarity_matrix = cosine_similarity(scaled_audio_features)
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(scaled_audio_features)
song_index = 0 
distances, indices = knn.kneighbors([scaled_audio_features[song_index]])
recommendations = indices.flatten()[1:]
for i in recommendations:
    print(df.loc[i, 'song_name'])

Kid Cudi (Remix)
Slidin
THROW DAT SET FT. B GIBS
Dead Body Vandal
Pop Out (feat. Lil Tjay)
Brand New
Opp Block Spinners
Xoxo
PATNAH DEM


In [ ]:
#Hierarchical Clustering
from sklearn.cluster import AgglomerativeClustering

audio_features = df1.iloc[:, 0:11].values
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

# Apply hierarchical clustering with 5 clusters
model = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
labels = model.fit_predict(scaled_audio_features)

# Get the songs in each cluster
for i in range(5):
    cluster_songs = df1.loc[labels == i, 'song_name'].values
    print(f'Cluster {i+1}: {cluster_songs}')


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


Cluster 1: ['Andromeda' 'Backpack' 'D(R)Own' ... 'Keep It Gangsta'
 "Thuggin Til I'm Gone" 'Love']
Cluster 2: ['Mercury: Retrograde' 'Symbiote'
 'ProductOfDrugs (Prod. The Virus and Antidote)' ... "Daddy's Girl" "'94"
 'Born 2 Live']
Cluster 3: ['Pathology' 'BRAINFOOD' 'Okay,ButThisIsTheLastTime' ...
 'Girls Need Love (with Drake) - Remix' 'Bad for You (feat. Nicki Minaj)'
 "When I'm Gone"]
Cluster 4: ['Coffin Wave' 'OhNo!' 'Claustrophobia' ... 'When the Sh-- Goes Down'
 'Deep Inside' 'Change']
Cluster 5: ['Sacrifice' 'Io sono qui' 'Hades' ... 'Life of a Sinner'
 "I'm Sorry Baby (Remix) [feat. Wadz & Talkbox Pewee]" "Can't Hold On"]


In [ ]:
#Density-Based Clustering
from sklearn.cluster import DBSCAN

audio_features = df1.iloc[:, 0:11].values
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

# Apply density-based clustering with a minimum cluster size of 5
model = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
labels = model.fit_predict(scaled_audio_features)

# Get the songs in each cluster
unique_labels = np.unique(labels)
for label in unique_labels:
    if label == -1:
        print(f'Noise: {np.sum(labels == label)} songs')
    else:
        cluster_songs = df1.loc[labels == label, 'song_name'].values
        print(f'Cluster {label}: {cluster_songs}')


Noise: 18296 songs
Cluster 0: ['Bi-Polar Freestyle' 'Took Her To The O' 'Free All The Opps'
 'Bi-Polar Freestyle' 'Thumpin']
Cluster 1: ['Misery Business' 'Pressure' 'Misery Business' "The Kids Aren't Alright"
 'Have Faith In Me' 'Smells Like Teen Spirit' 'Smells Like Teen Spirit']
Cluster 2: ['Move Along' 'Five Minutes to Midnight'
 "Damned If I Do Ya (Damned If I Don't)" 'Whoa Oh! (Me vs Everyone)'
 'Kiss Me' 'Up off the Ground' 'Move Along' 'Five Minutes to Midnight']
Cluster 3: ['Jamie All Over' 'Call Me Hopeless, but Not Romantic' 'Jamie All Over'
 'Letters To You' 'Jamie All Over' 'Jamie All Over' 'Letters To You']
Cluster 4: ['Dark Blue' 'My Friends Over You' 'Devil In Jersey City'
 'Calling All Cars' 'My Friends Over You']
Cluster 5: ['Three Cheers For Five Years' 'All Around Me' 'Take My Hand'
 'All Around Me' 'Drugs & Candy']
Cluster 6: ["Cute Without The 'E' (Cut From The Team)"
 'Cute Without the "E" (Cut From the Team)' 'Let Me Go'
 "Hate (I Really Don't Like You)"
 "Cute 

In [ ]:
#Spectral Clustering
from sklearn.cluster import SpectralClustering

audio_features = df1.iloc[:, 0:11].values
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

# Apply spectral clustering with 5 clusters
model = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', n_neighbors=10)
labels = model.fit_predict(scaled_audio_features)

# Get the songs in each cluster
for i in range(5):
    cluster_songs = df1.loc[labels == i, 'song_name'].values
    print(f'Cluster {i+1}: {cluster_songs}')


In [ ]:
#Fuzzy C-Means algorithm:
!pip install -U scikit-fuzzy
!pip install fuzzy-c-means


In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from fcmeans import FCM

audio_features = df1.iloc[:, 0:11].values
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

# Apply PCA to reduce the dimensionality of the features
pca = PCA(n_components=2)
pca_audio_features = pca.fit_transform(scaled_audio_features)

# Apply fuzzy clustering with 5 clusters and a fuzziness parameter of 2
fcm = FCM(n_clusters=5, m=2)
fcm.fit(pca_audio_features)
labels = fcm.predict(pca_audio_features)

# Get the songs in each cluster
for i in range(5):
    cluster_songs = df1.loc[labels == i, 'song_name'].values
    print(f'Cluster {i+1}: {cluster_songs}')


Cluster 1: ['Murder' 'OhNo!' 'Vengeance' ... 'Hold Tight' 'Mourn' 'Forever']
Cluster 2: ['Mercury: Retrograde' 'Venom' 'Andromeda' ... "It's LBC 2.0"
 "I'm Sorry Baby (Remix) [feat. Wadz & Talkbox Pewee]" "'94"]
Cluster 3: ['D(R)Own' "High 'N Mighty" 'BLUE JUICE' ... "Daddy's Girl" 'Born 2 Live'
 "Can't Hold On"]
Cluster 4: ['Symbiote' 'ProductOfDrugs (Prod. The Virus and Antidote)'
 'kamikaze (+ pulse)' ... 'Love' 'Deep Inside' 'Change']
Cluster 5: ['Pathology' 'Gatteka' 'BRAINFOOD' ... 'Gold'
 'Girls Need Love (with Drake) - Remix' 'All Day Chill']
